# Check synchronization of local and cloud sqlite databases

In [ ]:
import os

instance_name = f"lamindb-setup-ci-test-sqlite-sync-{os.environ['LAMIN_ENV']}"
!lamin load {instance_name}
!yes | lamin delete {instance_name}

In [ ]:
from lamindb_setup import init, settings
import os

In [ ]:
init(
    storage="s3://lndb-setup-ci",
    name=instance_name,
)

Get the paths to the cloud and local sqlite databases.

In [ ]:
sqlite_file = settings.instance._sqlite_file

In [ ]:
sqlite_file

Remote SQLite file does exists upon instance init:

In [ ]:
assert settings.instance._sqlite_file.exists()

Now mimic a new user who loads the instance (this runs 4s):

In [ ]:
settings.instance._update_local_sqlite_file()

Get the mere filepath of the local file, without any update:

In [ ]:
cache_file = settings.instance.storage.cloud_to_local_no_update(sqlite_file)

In [ ]:
cache_file

Delete the local sqlite file:

In [ ]:
cache_file.unlink()

In [ ]:
assert not cache_file.exists()

Update the local version of the sqlite file:

In [ ]:
settings.instance._update_local_sqlite_file()

In [ ]:
assert cache_file.exists()

If the local sqlite database is older than the cloud one, the cloud database replaces the local sqlite database file.

In [ ]:
cloud_mtime = sqlite_file.modified.timestamp()

In [ ]:
cloud_mtime

In [ ]:
os.utime(cache_file, times=(cloud_mtime - 1, cloud_mtime - 1))

In [ ]:
assert cache_file.stat().st_mtime < sqlite_file.modified.timestamp()

In [ ]:
settings.instance._update_local_sqlite_file()

In [ ]:
assert cache_file.stat().st_mtime == sqlite_file.modified.timestamp()

In [ ]:
!yes | lamin delete {instance_name}